In [10]:
import requests
import re
import json
import sqlite3

def scrape_election_data_from_view_source(state_name):
    url = f"https://www.nbcnews.com/politics/2024-elections/{state_name}-president-results"
    response = requests.get(url)
    json_pattern = re.compile(
            r'\[\{"name":"[^"]+","percentIn":\d+\.?\d*,"votes":\d+,"candidates":\[.*?\]\}\]'
        )
    match = json_pattern.search(response.text)
    data = json.loads(match.group(0))
    return data

def save_to_sqlite_single_table(db_name, state_name, data):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS election_results (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            state TEXT,
            county_name TEXT,
            percent_in REAL,
            total_votes INTEGER,
            candidate_name TEXT,
            candidate_party TEXT,
            candidate_votes INTEGER,
            candidate_percent_vote REAL
        )
    ''')

    for county in data:
        state = state_name
        county_name = county.get('name')
        percent_in = county.get('percentIn')
        total_votes = county.get('votes')

        for candidate in county.get('candidates', []):
            candidate_name = candidate.get('name')
            candidate_party = candidate.get('party')
            candidate_votes = candidate.get('votes')
            candidate_percent_vote = candidate.get('percentVote')

            cursor.execute('''
                INSERT INTO election_results (
                    state, county_name, percent_in, total_votes,
                    candidate_name, candidate_party, candidate_votes, candidate_percent_vote
                )
                VALUES (?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                state, county_name, percent_in, total_votes,
                candidate_name, candidate_party, candidate_votes, candidate_percent_vote
            ))
    conn.commit()
    conn.close()

def scrape_and_save_all_states_single_table():
    states = [
        "alabama", "alaska", "arizona", "arkansas", "california", "colorado",
        "connecticut", "delaware", "district-of-columbia", "florida", "georgia",
        "hawaii", "idaho", "illinois", "indiana", "iowa", "kansas", "kentucky",
        "louisiana", "maine", "maryland", "massachusetts", "michigan", "minnesota",
        "mississippi", "missouri", "montana", "nebraska", "nevada", "new-hampshire",
        "new-jersey", "new-mexico", "new-york", "north-carolina", "north-dakota",
        "ohio", "oklahoma", "oregon", "pennsylvania", "rhode-island",
        "south-carolina", "south-dakota", "tennessee", "texas", "utah", "vermont",
        "virginia", "washington", "west-virginia", "wisconsin", "wyoming"
    ]

    db_name = "election_results_new.db"

    for state in states:
        print(f"Scraping data for {state}...")
        data = scrape_election_data_from_view_source(state)
        if data:
            save_to_sqlite_single_table(db_name, state, data)
scrape_and_save_all_states_single_table()

Scraping data for alabama...
Scraping data for alaska...
Scraping data for arizona...
Scraping data for arkansas...
Scraping data for california...
Scraping data for colorado...
Scraping data for connecticut...
Scraping data for delaware...
Scraping data for district-of-columbia...
Scraping data for florida...
Scraping data for georgia...
Scraping data for hawaii...
Scraping data for idaho...
Scraping data for illinois...
Scraping data for indiana...
Scraping data for iowa...
Scraping data for kansas...
Scraping data for kentucky...
Scraping data for louisiana...
Scraping data for maine...
Scraping data for maryland...
Scraping data for massachusetts...
Scraping data for michigan...
Scraping data for minnesota...
Scraping data for mississippi...
Scraping data for missouri...
Scraping data for montana...
Scraping data for nebraska...
Scraping data for nevada...
Scraping data for new-hampshire...
Scraping data for new-jersey...
Scraping data for new-mexico...
Scraping data for new-york..

In [11]:
import pandas as pd
election_data = pd.read_csv("countypres_2000-2020.csv")

election_data_cleaned = election_data[
    ['year', 'state', 'state_po', 'county_name', 'county_fips', 'candidate', 
     'party', 'candidatevotes', 'totalvotes']
].dropna()

election_data_cleaned = election_data_cleaned.rename(columns={
    'state_po': 'state_abbreviation',
    'county_fips': 'fips_code',
    'candidatevotes': 'votes_candidate',
    'totalvotes': 'votes_total'
})

election_data_cleaned

,year,state,state_abbreviation,county_name,fips_code,candidate,party,votes_candidate,votes_total
0,2000,ALABAMA,AL,AUTAUGA,1001.0,AL GORE,DEMOCRAT,4942,17208
1,2000,ALABAMA,AL,AUTAUGA,1001.0,GEORGE W. BUSH,REPUBLICAN,11993,17208
2,2000,ALABAMA,AL,AUTAUGA,1001.0,RALPH NADER,GREEN,160,17208
3,2000,ALABAMA,AL,AUTAUGA,1001.0,OTHER,OTHER,113,17208
4,2000,ALABAMA,AL,BALDWIN,1003.0,AL GORE,DEMOCRAT,13997,56480
...,...,...,...,...,...,...,...,...,...
72612,2020,WYOMING,WY,WASHAKIE,56043.0,DONALD J TRUMP,REPUBLICAN,3245,4032
72613,2020,WYOMING,WY,WESTON,56045.0,JOSEPH R BIDEN JR,DEMOCRAT,360,3560
72614,2020,WYOMING,WY,WESTON,56045.0,JO JORGENSEN,LIBERTARIAN,46,3560
72615,2020,WYOMING,WY,WESTON,56045.0,OTHER,OTHER,47,3560


In [12]:
db_name = "election_results_new.db"
table_name = "cleaned_election_data"
conn = sqlite3.connect(db_name)
election_data_cleaned.to_sql(table_name, conn, if_exists="replace", index=False)
print(f"Cleaned data saved to the '{table_name}' table in the database '{db_name}'.")
conn.close()

Cleaned data saved to the 'cleaned_election_data' table in the database 'election_results_new.db'.


In [13]:
unemployment_data = pd.read_csv("Unemployment(UnemploymentMedianIncome).csv")
unemployment_cols = [col for col in unemployment_data.columns if 'Unemployment_rate_' in col]
unemp_long = unemployment_data.melt(
    id_vars=['FIPS_Code', 'State', 'Area_Name'],
    value_vars=unemployment_cols,
    var_name='year_var',
    value_name='unemp_rate'
)
unemp_long['year'] = unemp_long['year_var'].str.extract(r'(\d{4})').astype(int)
unemp_long = unemp_long.dropna(subset=['unemp_rate'])
unemp_long = unemp_long[(unemp_long['FIPS_Code'] != 0) & (~unemp_long['FIPS_Code'].astype(str).str.endswith('00'))]
unemp_long = unemp_long.rename(columns={'FIPS_Code': 'fips_code'})
unemp_long

,fips_code,State,Area_Name,year_var,unemp_rate,year
2,1001,AL,"Autauga County, AL",Unemployment_rate_2000,4.1,2000
3,1003,AL,"Baldwin County, AL",Unemployment_rate_2000,3.7,2000
4,1005,AL,"Barbour County, AL",Unemployment_rate_2000,5.6,2000
5,1007,AL,"Bibb County, AL",Unemployment_rate_2000,5.4,2000
6,1009,AL,"Blount County, AL",Unemployment_rate_2000,3.5,2000
...,...,...,...,...,...,...
75366,72145,PR,"Vega Baja Municipio, PR",Unemployment_rate_2022,7.4,2022
75367,72147,PR,"Vieques Municipio, PR",Unemployment_rate_2022,6.9,2022
75368,72149,PR,"Villalba Municipio, PR",Unemployment_rate_2022,10.4,2022
75369,72151,PR,"Yabucoa Municipio, PR",Unemployment_rate_2022,8.3,2022


In [14]:
table_name = "unemployment_data"
conn = sqlite3.connect(db_name)
unemp_long.to_sql(table_name, conn, if_exists="replace", index=False)
print(f"Unemployment data saved to the '{table_name}' table in the database '{db_name}'.")
conn.close()


Unemployment data saved to the 'unemployment_data' table in the database 'election_results_new.db'.


In [15]:
file_2000_2010 = 'population-2000-2010.csv'
file_2010_2020 = 'population-2010-2020.csv'
file_2020_2023 = 'population-2020-2023.csv'

pop_2000_2010 = pd.read_csv(file_2000_2010, encoding='ISO-8859-1')
pop_2010_2020 = pd.read_csv(file_2010_2020, encoding='ISO-8859-1')
pop_2020_2023 = pd.read_csv(file_2020_2023, encoding='ISO-8859-1')

columns_to_keep_2000 = ['STATE', 'COUNTY', 'STNAME'] + [f'POPESTIMATE{year}' for year in range(2000, 2011)]
columns_to_keep_2010 = ['STATE', 'COUNTY', 'STNAME'] + [f'POPESTIMATE{year}' for year in range(2011, 2021)]
columns_to_keep_2020 = ['STATE', 'COUNTY', 'STNAME'] + [f'POPESTIMATE{year}' for year in range(2021, 2023)]

pop_2000_2010_filtered = pop_2000_2010[columns_to_keep_2000]
pop_2010_2020_filtered = pop_2010_2020[columns_to_keep_2010]
pop_2020_2023_filtered = pop_2020_2023[columns_to_keep_2020]
pop_2000_2010_filtered = pop_2000_2010_filtered.reset_index(drop=True)
pop_2010_2020_filtered = pop_2010_2020_filtered.reset_index(drop=True)
pop_2020_2023_filtered = pop_2020_2023_filtered.reset_index(drop=True)

combined_df = pd.merge(
    pop_2000_2010_filtered,
    pop_2010_2020_filtered,
    on=['STATE', 'COUNTY', 'STNAME'],
    how='inner'
)

combined_df = pd.merge(
    combined_df,
    pop_2020_2023_filtered,
    on=['STATE', 'COUNTY', 'STNAME'],
    how='inner'
)

combined_df = combined_df[combined_df['COUNTY'] != 0]

combined_df['FIPS'] = combined_df['STATE'].astype(str).str.zfill(1) + combined_df['COUNTY'].astype(str).str.zfill(3)

combined_df = combined_df.reset_index(drop=True)

combined_df = combined_df.sort_values(by=['STATE', 'COUNTY']).reset_index(drop=True)

combined_df

,STATE,COUNTY,STNAME,POPESTIMATE2000,POPESTIMATE2001,POPESTIMATE2002,POPESTIMATE2003,POPESTIMATE2004,POPESTIMATE2005,POPESTIMATE2006,...,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016,POPESTIMATE2017,POPESTIMATE2018,POPESTIMATE2019,POPESTIMATE2020,POPESTIMATE2021,POPESTIMATE2022,FIPS
0,1,1,Alabama,44021,44889,45909,46800,48366,49676,51328,...,54922,54903,55302,55448,55533,55769,56145,59203,59726,1001
1,1,3,Alabama,141342,144875,147957,151509,156266,162183,168121,...,199306,203101,207787,212737,218071,223565,229287,239439,246531,1003
2,1,5,Alabama,29015,28863,28653,28594,28287,28027,27861,...,26768,26300,25828,25169,24887,24657,24589,24533,24700,1005
3,1,7,Alabama,19913,21028,21199,21399,21721,22042,22099,...,22541,22553,22590,22532,22300,22313,22136,22359,21986,1007
4,1,9,Alabama,51107,51845,52551,53457,54124,54624,55485,...,57536,57535,57487,57801,57770,57840,57879,59079,59516,1009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3126,56,37,Wyoming,37552,36899,37428,37450,38026,38739,39749,...,44996,44780,44319,43663,43188,42917,42673,41626,41374,56037
3127,56,39,Wyoming,18381,18653,18837,19066,19467,19632,20014,...,22801,23083,23255,23383,23261,23385,23497,23605,23297,56039
3128,56,41,Wyoming,19666,19413,19587,19480,19470,19494,19709,...,20835,20777,20711,20449,20299,20196,20215,20681,20727,56041
3129,56,43,Wyoming,8252,8068,7988,7976,7960,8022,7979,...,8277,8282,8180,8013,7886,7824,7760,7719,7724,56043


In [16]:
table_name = "population_data"
conn = sqlite3.connect(db_name)
combined_df.to_sql(table_name, conn, if_exists="replace", index=False)
print(f"Population data saved to the '{table_name}' table in the database '{db_name}'.")
conn.close()

Population data saved to the 'population_data' table in the database 'election_results_new.db'.


In [17]:
file_path = "Education.xlsx"
education_df = pd.read_excel(file_path, engine='openpyxl', header=3)
education_df = education_df.rename(columns={'FIPS Code': 'fips_code'})
bachelors_cols = [
    "Percent of adults with a bachelor's degree or higher, 1990",
    "Percent of adults with a bachelor's degree or higher, 2000",
    "Percent of adults with a bachelor's degree or higher, 2008-12",
    "Percent of adults with a bachelor's degree or higher, 2018-22"
]
hs_cols = [
    "Percent of adults with a high school diploma only, 1990",
    "Percent of adults with a high school diploma only, 2000",
    "Percent of adults with a high school diploma only, 2008-12",
    "Percent of adults with a high school diploma only, 2018-22"
]

edu_cols = ['fips_code'] + bachelors_cols + hs_cols
education_df = education_df[edu_cols]
education_df = education_df.rename(columns={
    "Percent of adults with a bachelor's degree or higher, 1990": "bach_1990",
    "Percent of adults with a bachelor's degree or higher, 2000": "bach_2000",
    "Percent of adults with a bachelor's degree or higher, 2008-12": "bach_2008_12",
    "Percent of adults with a bachelor's degree or higher, 2018-22": "bach_2018_22",
    "Percent of adults with a high school diploma only, 1990": "hs_1990",
    "Percent of adults with a high school diploma only, 2000": "hs_2000",
    "Percent of adults with a high school diploma only, 2008-12": "hs_2008_12",
    "Percent of adults with a high school diploma only, 2018-22": "hs_2018_22"
})
education_df = education_df[
    (education_df['fips_code'] != 0) & 
    (~education_df['fips_code'].astype(str).str.endswith('00'))
]
education_df = education_df.reset_index(drop=True)
education_df

,fips_code,bach_1990,bach_2000,bach_2008_12,bach_2018_22,hs_1990,hs_2000,hs_2008_12,hs_2018_22
0,1001,14.5,18.0,21.707831,29.558575,32.0,33.8,33.786706,31.146113
1,1003,16.8,23.1,27.741591,32.561579,31.8,29.6,28.816463,27.775383
2,1005,11.8,10.9,14.524286,11.881188,27.1,32.4,33.294700,36.814710
3,1007,4.7,7.1,8.996005,10.919937,33.8,35.7,41.609744,40.879121
4,1009,7.0,9.6,12.381469,14.741407,34.7,36.0,36.157404,35.313717
...,...,...,...,...,...,...,...,...,...
3232,72145,8.6,14.4,17.584094,25.094723,21.3,20.6,15.068387,29.239957
3233,72147,6.7,10.1,10.140029,14.542131,20.2,26.5,33.719620,46.418481
3234,72149,10.2,12.8,15.207082,21.621274,21.3,25.0,35.959532,34.408602
3235,72151,8.7,12.1,12.912144,18.433569,20.0,23.5,30.389885,25.887896


In [18]:
table_name = "education_data"
conn = sqlite3.connect(db_name)
education_df.to_sql(table_name, conn, if_exists="replace", index=False)
print(f"Education data saved to the '{table_name}' table in the database '{db_name}'.")
conn.close()

Education data saved to the 'education_data' table in the database 'election_results_new.db'.
